# San Francisco Bike Share Prediction Model 

I have done a previous analysis on bike share data of San Francisco where I extracted trends and patterns from the data. I found a different set of ride share data that that has a few new columns, which will come in handy when creating a prediction model. I want to create a model that will predict how many bike rides are taken on a daily basis, as a continuation of the analysis. Highly recommed taking a look at it, https://github.com/medmorales/Data-Exploration/blob/main/Lyft%20Analysis/BikeShareAnalysis.ipynb.



## Packages

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

/Users/miguelmorales/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Loading in the data

In [ ]:
#station =  pd.read_csv('station.csv')
#status =  pd.read_csv('status.csv')
#trip =  pd.read_csv('trip.csv')
#weather =  pd.read_csv('weather.csv')

## Trips DataFrame
The trips file has the bulk of our data. This is very similar to the data that I did an exploratory analysis on as mentioned at the top of this file. I highly recommend taking a look at that first. https://github.com/medmorales/Data-Exploration/blob/main/Lyft%20Analysis/BikeShareAnalysis.ipynb

In [ ]:
trip =  pd.read_csv('trip.csv')

In [ ]:
trip.head()

In [ ]:
trip.dtypes

In [ ]:
trip.describe()

In [ ]:
trip.nunique()

Here I just wanted to see the number of unique values in the data frame to gain a better understanding of the # of stations, subscription types, etc..

In [ ]:
#### Missing Data 
np.round(trip.isnull().sum()/len(trip)*100, decimals =2)

As we can see, the zip code column has about 1% of it's data missing. I think it'll be intersting to see if there is a pattern for these missing zip codes so that we can populated the missing values. 

My mind questions if they are more isolated to only some stations, possibly to a subscription type or a start/end station maybe? Let's take a closer look at these rows...

In [ ]:
# isolating rows with missing zip codes
missing_zip = trip[trip['zip_code'].isnull()]

### Duration: Let's explore the duration of the trips. 
When looking at the output after using the describe function earlier, there seems to be some questionable and unrealistic ride durations, so we'll take a closer look.


In [ ]:
# The duration uses the units seconds, I want to change this to minutes for clarity.
trip['duration']= trip['duration']/60

In [ ]:
trip.duration.describe()

From this output and our original analysis, we know that there are outliers in the data. Especially seeing the trip with the highest duration being one that lasted 287,840 minutes. I do no think it is realistic for someone to have taken a 287,840 minute bike ride, so the outliers might be the result of recording errors. Therefore, I will look into removing them prior to running the model. I am going to continue looking at the different data thats available, but will be thinking about my options for these outliers, maybe reducing the amount of outliers by creating a cutoff. With outliers like this,this data does not seem normal, complicating what we can do to identify outliers. 

### Start Date
The start_date column will be essential to our model as it will be the date at which we will use to count the number of daily trips, our Dependent (x) variable that we will be predicting.  

In [ ]:
# transform object type to datetime
trip.start_date = pd.to_datetime(trip.start_date, format='%m/%d/%Y %H:%M')

### Trip Duration Outlier Detection and Removal 

For the most part, when detecting and removing outliers, we use one of two main methods, using the Z-Score or Interquartile Range.

##### Z-Score

The z-score is the number of standard deviations that a value of an observation is above or below the mean value of the observations that are being measured. 

The formula for the z-szore is given by: $ z = (x - mean) / sd $

In order to use the z-score we need the assumption that the data has a Gaussian Distribution, also known as a Normal distribution. 

In finding the z-score, we rescale and center the data, and look for data points that are too far away from the mean. In this case we will use a threshold of 3 standard deviations away from the new mean. This is pretty standard way of defining what an outlier means when using the z-score method. 

In [ ]:
# Removing Outliers
outliers_removed = trip[np.abs(stats.zscore(trip["duration"])) < 3]

In [ ]:
outliers_removed.shape

In [ ]:
outliers_removed.describe()

Now the max is 1,130 minutes. This looks a lot more reasonable before, but still surprising that a bike ride would last over 18 hours total. 

In [ ]:
# Distribution of durations. 
sns.displot(x= np.clip(trip.duration, 0,40), data = trip, bins = 200, kde= True)
# sns.displot('duration', data = trip, bins = 20, kde= True)
plt.title('Duration Distribution')
plt.show()

Either way, the distribution of the this data, even after we shorten the x-axis to a max of 4 minutes is left skewed, so our data does not meet the assumptions of normaility for using the z-score in first place. 

### Extracting Date and Caluclating Daily Counts

In [ ]:
# Extracting date from start time
trip["only_date"] = [d.date() for d in trip["start_date"]]

In [ ]:
# Calculating daily trips and creating new dataframe. 
daily_count = pd.DataFrame(trip['only_date'].value_counts().reset_index())
daily_count.columns = ['date', 'num_trips']

In [ ]:
# sort the dataframe by date
train_data = daily_count.sort_values('date')
train_data.reset_index(drop = True, inplace = True)

In [ ]:
print('The trips data has a date range between ' + str(train_data.date.min()) 
      + ' and '+ str(train_data.date.max()) + '.')

In [ ]:
train_data.head()

Now, we have successfuly calculated the number of bike rides per day given the date range in the Trip's dataset. This will be the focus of our model, predicting the number of trips, given the date. 

## Station DataFrame

In [ ]:
station =  pd.read_csv('station.csv')
station.head()

In [ ]:
station.dtypes

Looking at the station dataframe, I think there is a great way to make use of it. Stations have different installation dates, possibly meaning that the amount of bikes available to take for ride share has changed over time because they have expanded their business. So, an important variable in predicting the number of bike rides by date has to be the number of open stations, docks, and bikes at the time.  

So, lets calculate the number of docks at each date and combine that to our train_data set which is our modeling dataset. 

In [ ]:
station.describe()

In [ ]:
station.nunique()

#### Transforming and Extracting the Installation Date

In [ ]:
# Extracting date from instillation time
# transform object type to datetime
station.installation_date = pd.to_datetime(station.installation_date, format='%m/%d/%Y')
station["installation_date"] = [d.date() for d in station["installation_date"]]

In [ ]:
station['installation_date']

In [ ]:
print('The stations data has a date range between ' + str(station.installation_date.min()) 
      + ' and '+ str(station.installation_date.max()) + '.')

The date range of the station installation dates basically fall between the the data range of our trip data. So this will be a great data point to have. 

Now, going back to what was interesting about this dataframe, calculating the dock counts for each date. So, lets get this done. 

In [ ]:
# for each data in train_data, we will calculate the sum of dock_counts
avail_docks=[]
for date in train_data.date:
    num = 0
    num = station.loc[station['installation_date'] <= date, 'dock_count'].sum()
    avail_docks.append(num)    

Now, we have succefully created a list of the number of available docks for each date in our dataframe. We'll go ahead and merge this list into our train dataset.

In [ ]:
train_data['dock_count'] = avail_docks

In [ ]:
train_data.head()

This is great, we are making progress!

## Weather Data

In [ ]:
weather =  pd.read_csv('weather.csv')
weather.head()

This weather dataset has many good data points for specific dates. We can definitly use this in our model. 

In [ ]:
weather.shape

In [ ]:
weather.dtypes

In [ ]:
# calculating percentage of null values in columns
round(weather.isnull().sum()/(len(weather))*100, 2)

There seems to be two columns that are missing a significant amount of values.

In [ ]:
# again we need to change the data type of the date column. 
weather.date = pd.to_datetime(weather.date, format='%m/%d/%Y')

In [ ]:
weather.events.value_counts()

We can see here that there are two rain values. Let's convert all 'rain' to 'Rain' for more consistency. 

In [ ]:
weather.loc[weather['events'] == 'rain', 'events'] = 'Rain'

In [ ]:
weather.events.value_counts()

Upon trying to fit the regression model, I came across an error in the 'Percipitation_inches' column. Most values are in decimals, however, there is also a value of 'T' that is signficantly prominent throughout the data. We need to understand what this value means and think about replacing it with a number. 

In [ ]:
pd.set_option('display.max_rows', 500)
weather.head(200)

It looks like most T's that are in the data have an 'event' of either Rain or Fog-Rain. Therefore, we should replace the T's in the data with a percitpitation amount. What amount should we use? Well I think we have two options, either replacing it with the mean or median of the percirpitation amount. Since it is possible that heavy rainfall can occur, I do not think taking the mean value of precipitation would be robust, instead we will use the median. 

In [ ]:
no_inches = ['T', '0']
median= weather[~weather['precipitation_inches'].isin(no_inches)]
median.precipitation_inches.median()

In [ ]:
weather.loc[weather['precipitation_inches'] == 'T', 'precipitation_inches'] = '0.08'

We have successfully replaced all 'T' values with the median percipitation amount. Now all we have to do is convert this column from an object to float.

In [ ]:
weather['precipitation_inches'] = weather.precipitation_inches.astype('float')

In [ ]:
weather.dtypes

In [ ]:
weather.date

Looking at the date counts, we can see that there are a total of 733 unique dates and a total of 3665 total rows. Which should be enough to confirm that each date appears a total of five times. This is good informtion to know!

In [ ]:
weather.zip_code.value_counts()

I initially thought that the weather data point was taken five seperate times per day, but now it looks like the weather data has a daily entry for the 5 unique zip_codes that are present. These zip codes stretch from San Francisco to San Jose. Due to their close proximity, I think it is reasonable to suggest that they experience very similar weather, so let's just use one zip codes data for our weather subset in our model. 

In [ ]:
trip.zip_code.value_counts()

Looking at the top five, zip code counts from the trips dataset, they are all from San Francisco. So, I think it would be easiest to use 94107's (located in San Francisco) data from the weather dataset to supply us with our weather data points. 

In [ ]:
weather = weather.loc[weather['zip_code'] == 94107]

In [ ]:
round(weather.isnull().sum()/(len(weather))*100, 2)

In [ ]:
weather.drop('max_gust_speed_mph', axis=1, inplace=True)
weather.drop('events', axis=1, inplace=True)

In [ ]:
weather.dtypes

In [ ]:
train_data.head()

In [ ]:
train_data.dtypes

In [ ]:
train_data.date = pd.to_datetime(train_data.date, format='%Y-%m-%d')

In [ ]:
train_data = train_data.merge(weather, on='date', how='left')

In [ ]:
train_data.head()

In [4]:
model_data.to_csv('model_data.csv')

NameError: name 'model_data' is not defined

## Creating Model

In [47]:
train =  pd.read_csv('train.csv')

In [48]:
train.head()

,Unnamed: 0,date,num_trips,dock_count,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,...,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees,zip_code
0,0,2013-08-29,748,1150,74.0,68.0,61.0,61.0,58.0,56.0,...,29.97,10.0,10.0,10.0,23.0,11.0,0.0,4.0,286.0,94107
1,1,2013-08-30,714,1150,78.0,69.0,60.0,61.0,58.0,56.0,...,29.93,10.0,10.0,7.0,29.0,13.0,0.0,2.0,291.0,94107
2,2,2013-08-31,640,1150,71.0,64.0,57.0,57.0,56.0,54.0,...,29.92,10.0,10.0,10.0,26.0,15.0,0.0,4.0,284.0,94107
3,3,2013-09-01,706,1150,74.0,66.0,58.0,60.0,56.0,53.0,...,29.91,10.0,10.0,10.0,25.0,13.0,0.0,4.0,284.0,94107
4,4,2013-09-02,661,1150,75.0,69.0,62.0,61.0,60.0,58.0,...,29.90,10.0,10.0,6.0,23.0,12.0,0.0,6.0,277.0,94107


In [49]:
train.date = pd.to_datetime(train.date, format='%Y-%m-%d')

In [50]:
train.dtypes

Unnamed: 0                                 int64
date                              datetime64[ns]
num_trips                                  int64
dock_count                                 int64
max_temperature_f                        float64
mean_temperature_f                       float64
min_temperature_f                        float64
max_dew_point_f                          float64
mean_dew_point_f                         float64
min_dew_point_f                          float64
max_humidity                             float64
mean_humidity                            float64
min_humidity                             float64
max_sea_level_pressure_inches            float64
mean_sea_level_pressure_inches           float64
min_sea_level_pressure_inches            float64
max_visibility_miles                     float64
mean_visibility_miles                    float64
min_visibility_miles                     float64
max_wind_Speed_mph                       float64
mean_wind_speed_mph 

In [51]:
train['day_of_week'] = train.date.dt.weekday

In [54]:
train.dtypes

Unnamed: 0                                 int64
date                              datetime64[ns]
num_trips                                  int64
dock_count                                 int64
max_temperature_f                        float64
mean_temperature_f                       float64
min_temperature_f                        float64
max_dew_point_f                          float64
mean_dew_point_f                         float64
min_dew_point_f                          float64
max_humidity                             float64
mean_humidity                            float64
min_humidity                             float64
max_sea_level_pressure_inches            float64
mean_sea_level_pressure_inches           float64
min_sea_level_pressure_inches            float64
max_visibility_miles                     float64
mean_visibility_miles                    float64
min_visibility_miles                     float64
max_wind_Speed_mph                       float64
mean_wind_speed_mph 

In [59]:
train.loc[train['day_of_week'].isin([0,1,2,3,4]), 'weekend'] = 0
train.loc[train['day_of_week'].isin([5,6]), 'weekend'] = 1

In [60]:
train

,Unnamed: 0,date,num_trips,dock_count,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,precipitation_inches,cloud_cover,wind_dir_degrees,zip_code,day_of_week,weekend
0,0,2013-08-29,748,1150,74.0,68.0,61.0,61.0,58.0,56.0,...,10.0,10.0,23.0,11.0,0.00,4.0,286.0,94107,3,0
1,1,2013-08-30,714,1150,78.0,69.0,60.0,61.0,58.0,56.0,...,10.0,7.0,29.0,13.0,0.00,2.0,291.0,94107,4,0
2,2,2013-08-31,640,1150,71.0,64.0,57.0,57.0,56.0,54.0,...,10.0,10.0,26.0,15.0,0.00,4.0,284.0,94107,5,1
3,3,2013-09-01,706,1150,74.0,66.0,58.0,60.0,56.0,53.0,...,10.0,10.0,25.0,13.0,0.00,4.0,284.0,94107,6,1
4,4,2013-09-02,661,1150,75.0,69.0,62.0,61.0,60.0,58.0,...,10.0,6.0,23.0,12.0,0.00,6.0,277.0,94107,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,728,2015-08-27,1443,1236,84.0,72.0,59.0,58.0,55.0,46.0,...,10.0,10.0,17.0,6.0,0.00,2.0,272.0,94107,3,0
729,729,2015-08-28,1220,1236,91.0,77.0,62.0,62.0,58.0,53.0,...,10.0,10.0,24.0,8.0,0.00,4.0,291.0,94107,4,0
730,730,2015-08-29,342,1236,76.0,71.0,65.0,66.0,62.0,57.0,...,9.0,3.0,21.0,11.0,0.08,6.0,260.0,94107,5,1
731,731,2015-08-30,331,1236,78.0,70.0,61.0,60.0,59.0,56.0,...,10.0,10.0,20.0,11.0,0.00,3.0,276.0,94107,6,1


# 

In [61]:
train.drop('zip_code', axis=1, inplace=True)

In [62]:
train.drop('date', axis=1, inplace=True)

In [63]:
train.drop('Unnamed: 0', axis=1, inplace=True)

In [64]:
# checking for percentage of null values
np.round(train.isnull().sum()/len(train)*100, decimals =2)

num_trips                         0.0
dock_count                        0.0
max_temperature_f                 0.0
mean_temperature_f                0.0
min_temperature_f                 0.0
max_dew_point_f                   0.0
mean_dew_point_f                  0.0
min_dew_point_f                   0.0
max_humidity                      0.0
mean_humidity                     0.0
min_humidity                      0.0
max_sea_level_pressure_inches     0.0
mean_sea_level_pressure_inches    0.0
min_sea_level_pressure_inches     0.0
max_visibility_miles              0.0
mean_visibility_miles             0.0
min_visibility_miles              0.0
max_wind_Speed_mph                0.0
mean_wind_speed_mph               0.0
precipitation_inches              0.0
cloud_cover                       0.0
wind_dir_degrees                  0.0
day_of_week                       0.0
weekend                           0.0
dtype: float64

In [65]:
train.dtypes

num_trips                           int64
dock_count                          int64
max_temperature_f                 float64
mean_temperature_f                float64
min_temperature_f                 float64
max_dew_point_f                   float64
mean_dew_point_f                  float64
min_dew_point_f                   float64
max_humidity                      float64
mean_humidity                     float64
min_humidity                      float64
max_sea_level_pressure_inches     float64
mean_sea_level_pressure_inches    float64
min_sea_level_pressure_inches     float64
max_visibility_miles              float64
mean_visibility_miles             float64
min_visibility_miles              float64
max_wind_Speed_mph                float64
mean_wind_speed_mph               float64
precipitation_inches              float64
cloud_cover                       float64
wind_dir_degrees                  float64
day_of_week                         int64
weekend                           

In [66]:
y = train.num_trips.values

In [67]:
X = train.drop(['num_trips'], axis = 1).values

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 2)

In [69]:
regr = LinearRegression()

In [70]:
regr.fit(X_train, y_train)

LinearRegression()

In [71]:
print(regr.score(X_test, y_test))

0.722123981573403
